In [1]:
class Station(object):
    """The Station class represents a single name station on a subway line."""

    def __init__(self, name):
        """Every Station object has a name attribute"""
        self.name = name

    def __eq__(self, obj):
        """Equality of Stations object depends on the lowercase version of 
        their name"""
        if not isinstance(obj, Station):
            return False
        result = (self.name.lower() == obj.name.lower())
        return result

    # Since we have overidden the __eq__ method and we want to use a station 
    # object as a key to dictionary (a mapping type) so we need to
    # overide the __hash__ method.
    def __hash__(self):
        """A Station object's hash code depends on the hash code of
          the lower case version of its name"""
        return self.name.lower().__hash__()

    # The __str__ method returns a human-readable string representation. 
    def __str__(self):
        return f'{self.name}'

    # The __repr__ method returns a machine-readable string representation.
    __repr__ = __str__


class Connection(object):
    """The Connection class represents a connection between two subways
    stations along a particular subway line."""

    def __init__(self, station1, station2, line):
        """ Every Conneciton object has two stations and the name 
        of the line"""
        self.station1 = station1
        self.station2 = station2
        self.line = line

    def __eq__(self, obj):
        """Equality of Connection objects depends on the equality
        of there attribuites: station1, station2, line."""
        if not isinstance(obj, Connection):
            return False
        result1 = self.station1 == obj.station1
        result2 = self.station2 == obj.station2
        result3 = self.line == obj.line
        return (result1 and result2 and result3)

    def __str__(self):
        """String representaiton of a Connection object"""
        return f'{self.station1},{self.station2},{self.line}'

    __repr__ = __str__


class Subway(object):
    """The Subway class represents a subway line with stations 
    and connections bewteen those stations"""

    def __init__(self):
        """Every Subway object has a colleciton of stations and a 
        list of connecitons"""
        self.stations = []
        self.connections = []
        self.network = {}

    def add_station(self, name):
        """If we have never seen the specficed station name before,
        then we add it to our collections of station object"""
        if not self.has_station(name):
            self.stations.append(Station(name))

    def has_station(self, name):
        """Returns True if we have a station with the specified name."""
        temp_station = Station(name)
        return temp_station in self.stations

    def has_connection(self, name1, name2, line):
        """Returns True if we have a connection with specified attributes"""
        temp_station1 = Station(name1)
        temp_station2 = Station(name2)
        temp_connection = Connection(temp_station1, temp_station2, line)
        return temp_connection in self.connections

    def get_connection(self, station1, station2):
        """Returns the connection if exists, else return none"""
        for connection in self.connections:
            if station1 == connection.station1 and \
               station2 == connection.station2:
                return connection
        return None

    # Discovered/Added when creating SubwayLoader class.
    def add_connection(self, name1, name2, line):
        """Add a connection going in both directions to the subway as 
        long as specified names reference an existing station"""
        if self.has_station(name1) and self.has_station(name2):
            station1 = Station(name1)
            station2 = Station(name2)
            connection1 = Connection(station1, station2, line)
            connection2 = Connection(station2, station1, line)
            if connection1 not in self.connections:
                self.connections.append(connection1)
                self.add_to_network(station1, station2)
            if connection2 not in self.connections:
                self.connections.append(connection2)
                self.add_to_network(station2, station1)

    def add_to_network(self, station1, station2):
        """Create a network of neighbouring stations"""
        if station1 in self.network.keys():
            stations = self.network[station1]
            if station2 not in stations:
                stations.append(station2)
        else:
            self.network[station1] = [station2]

    def get_directions(self, name1, name2):
        """Returns a route between two stations.  Assumes a connected, closed 
        chain of stations.  If both stations exists, then a route exists.
        If route is empty, then one or both stations do not exist."""

        # Do we have the stations in our list?
        if not self.has_station(name1) or not self.has_station(name2):
            return []

        # Stations exist, lets create the station objects and some empty
        # lists and a dictionary.  The Data structures (list and dictionary)
        # will be used to add information about if we can reach the staiton
        # as we build the route.  The list 'previous' is used to allow
        # us to reconstruct the connections.
        station1 = Station(name1)
        station2 = Station(name2)
        route = []
        reachable = []
        previous = {}

        # Check for the simple case where only need to travel one station.
        # If the destination is one station away then we are done. Append 
        # the connection to the route and stop executing instructions
        # that is, retrun from the method.  Otherwise, keep track of
        # what is reachable, and the previous station.
        neighbours = self.network[station1]
        for station in neighbours:
            if station == station2: 
                route.append(self.get_connection(station1, station2))
                return route
            else:
                reachable.append(station)
                previous[station] = station1

        # Okay, so the destination is more than one station away. 
        # Keep a list of stations to check from our current location
        next_stations = []
        next_stations.extend(neighbours)
        current_station = station1

        found_it = False  # allow early exit once route found

        # Try every station in our list (database)
        for i in range(1, len(self.stations)):
            if found_it:
                break
            tmp_next_stations = []
            for station in next_stations:
                if found_it:
                    break
                reachable.append(station)
                current_station = station
                current_neighbours = self.network[current_station]
                for neighbour in current_neighbours:
                    if found_it:
                        break
                    if neighbour == station2:
                        reachable.append(neighbour)
                        previous[neighbour] = current_station
                        found_it = True
                    elif neighbour not in reachable:
                        reachable.append(neighbour)
                        tmp_next_stations.append(neighbour)
                        previous[neighbour] = current_station
            next_stations = tmp_next_stations

        # Okay, we have found a path. Build a list of connections from the path.  
        # A connection object provides the direction of travel and the current line.
        # With this infromation, we can print which station to travel to
        # and if we are at a intersection, which line to change to. The way
        # we do this is in the method get_direciton()
        #
        # Initialise loop variables
        keep_looping = True
        key_station = station2
        station = None

        # Loop over path, adding connections
        while keep_looping:
            station = previous[key_station]
            route.insert(0, self.get_connection(station, key_station))
            if station1 == station:
                keep_looping = False
            key_station = station

        return route


class SubwayLoader():
    """Load a list of stations and the lines"""

    def __init__(self):
        self.subway = Subway()

    def load_from_file(self, station_file):
        """Load stations and subway-lines from file"""
        with open(station_file, mode='r') as data:
            self.load_stations(data)
            self.load_line(data)

    def load_stations(self, data):
        """Each station is on its own line.  The list of stations ends at the 
        first blank line""" 
        station = data.readline().strip()
        while station != "":
            self.subway.add_station(station)
            station = data.readline().strip()

    def load_line(self, data):
        """The first line of the block is a the subway-line name, after that
        each line contains a station."""
        line = data.readline().strip()
        while line != "":
            self.load_connection(data, line)
            line = data.readline().strip()

    def load_connection(self, data, line):
        """Read the next two stations and make a connection in both directions.
        The line-circuit is complete one read a blank line"""
        station1 = data.readline().strip()
        station2 = data.readline().strip()
        while station2 != "":
            self.subway.add_connection(station1, station2, line)
            station1 = station2
            station2 = data.readline().strip()


# Static Methods
def print_directions(route):
    """Create a text description of the subway route"""
    connection = route[0]
    station1 = connection.station1
    station2 = connection.station2
    current_line = connection.line
    previous_line = current_line

    print(f'Start out at {station1.name}.')
    print(f'Get on the {current_line} heading towards {station2.name}')

    for connection in route[1:]:
        current_line = connection.line
        station1 = connection.station1
        station2 = connection.station2
        if current_line == previous_line:
            print(f'Continue past {station1.name}...')
        else:
            print(f'When you get to {station1.name}, get off the {previous_line}.')
            print(f'Switch over to the {current_line}, heading towards {station2.name}.')
            previous_line = current_line

    print(f'Get off at {station2.name} and enjoy yourself!')


def str_directions(route):
    """Create a text description of the subway route"""
    connection = route[0]
    station1 = connection.station1
    station2 = connection.station2
    current_line = connection.line
    previous_line = current_line

    directions = f'Start out at {station1.name}. \n'
    directions += f'Get on the {current_line} heading towards {station2.name}\n'

    for connection in route[1:]:
        current_line = connection.line
        station1 = connection.station1
        station2 = connection.station2
        if current_line == previous_line:
            directions += f'Continue past {station1.name}...\n'
        else:
            directions += f'When you get to {station1.name}, get off the {previous_line}.\n'
            directions += f'Switch over to the {current_line}, heading towards {station2.name}.\n'
            previous_line = current_line

    directions += f'Get off at {station2.name} and enjoy yourself!\n'
    return directions

In [2]:
loader = SubwayLoader()
loader.load_from_file("objectville_subway.txt")
subway = loader.subway
print_directions(subway.get_directions("AJAX Rapids", "JavaRanch"))

Start out at Ajax Rapids.
Get on the Booch Line heading towards UML Walk
When you get to UML Walk, get off the Booch Line.
Switch over to the Wirfs-Brock Line, heading towards The Tikibean Lounge.
Continue past The Tikibean Lounge...
Continue past Head First Lounge...
Continue past Objectville Diner...
When you get to Servlet Springs, get off the Wirfs-Brock Line.
Switch over to the Jacobson Line, heading towards Mighty Gumball, Inc..
Continue past Mighty Gumball, Inc....
Get off at JavaRanch and enjoy yourself!
